In [2]:
# Importing the required libraries
import pandas as pd
import numpy as np

In [3]:
# Importing the dataset 
weather = pd.read_csv("../data/weather/weather.csv")
weather['timestamp'] = pd.to_datetime(weather['timestamp'])

### Missing Values 

In [4]:
def determine_column_counts(weather):
    columns = weather.columns
    data_counts = pd.DataFrame(columns = ['column_name','column_type','num_rows','distinct_data', 'distinct_percent', 'missing_data', 'missing_percentage'])
    
    #For each column, calculate number and percentage of missing rows and distinct counts
    for col in columns:
        col_vals = weather[col]
        total_data = len(col_vals)
        col_missing = sum(col_vals.isnull())
        missing_percent = round((col_missing / len(weather[col])) * 100 , 3)
        col_distinct = (col_vals.nunique())
        distinct_percent = round((col_distinct / len(weather[col])) * 100 , 3)

        data_counts.loc[len(data_counts)] = [col, col_vals.dtype, total_data, col_distinct, distinct_percent, col_missing, missing_percent]
        
    return(data_counts)

missing_data = determine_column_counts(weather)
missing_data

,column_name,column_type,num_rows,distinct_data,distinct_percent,missing_data,missing_percentage
0,timestamp,datetime64[ns],331166,17544,5.298,0,0.000
1,site_id,object,331166,19,0.006,0,0.000
2,airTemperature,float64,331166,646,0.195,128,0.039
3,cloudCoverage,float64,331166,10,0.003,170987,51.632
4,dewTemperature,float64,331166,553,0.167,328,0.099
5,precipDepth1HR,float64,331166,167,0.050,133186,40.217
6,precipDepth6HR,float64,331166,168,0.051,313004,94.516
7,seaLvlPressure,float64,331166,756,0.228,21624,6.530
8,windDirection,float64,331166,59,0.018,13005,3.927
9,windSpeed,float64,331166,73,0.022,574,0.173


In [5]:
# Impute Based on Season 
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

# Add Season Column 
weather['season'] = weather['timestamp'].dt.month.apply(get_season)

In [6]:
weather.columns

Index(['timestamp', 'site_id', 'airTemperature', 'cloudCoverage',
       'dewTemperature', 'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure',
       'windDirection', 'windSpeed', 'season'],
      dtype='object')

Before we imputute we will split the data into test and train (2016 and 2017 respectively) since we must impute 2017 values based on historical values

In [7]:
weather['timestamp'] = pd.to_datetime(weather['timestamp'])

# Define the function for determining seasons
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

# Apply the season function to create a 'season' column
weather['season'] = weather['timestamp'].dt.month.apply(get_season)

# Split the data into 2016 and 2017
weather_2016 = weather[weather['timestamp'].dt.year == 2016]
weather_2017 = weather[weather['timestamp'].dt.year == 2017]

# List of columns to impute
weather_columns = [
    'airTemperature', 'cloudCoverage', 'dewTemperature',
    'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure',
    'windDirection', 'windSpeed'
]

# Calculate seasonal means from 2016 data
seasonal_means_2016 = {column: weather_2016.groupby('season')[column].mean() for column in weather_columns}

# Function to impute missing values using the seasonal means
def impute_with_seasonal_means(df, seasonal_means):
    for column, means in seasonal_means.items():
        for season, mean in means.items():
            df.loc[(df['season'] == season) & (df[column].isnull()), column] = mean
    return df

# Impute missing values in both datasets
weather_2016_imputed = impute_with_seasonal_means(weather_2016.copy(), seasonal_means_2016)
weather_2017_imputed = impute_with_seasonal_means(weather_2017.copy(), seasonal_means_2016)

# Combine the datasets back into one:
weather_imputed = pd.concat([weather_2016_imputed, weather_2017_imputed], ignore_index=True)

# Show the head of the imputed dataset
weather_imputed.head()

,timestamp,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed,season
0,2016-01-01 00:00:00,Panther,19.4,2.026526,19.4,0.0,11.329319,1017.671008,0.0,0.0,Winter
1,2016-01-01 01:00:00,Panther,21.1,6.000000,21.1,-1.0,11.329319,1019.400000,0.0,0.0,Winter
2,2016-01-01 02:00:00,Panther,21.1,2.026526,21.1,0.0,11.329319,1018.800000,210.0,1.5,Winter
3,2016-01-01 03:00:00,Panther,20.6,2.026526,20.0,0.0,11.329319,1018.100000,0.0,0.0,Winter
4,2016-01-01 04:00:00,Panther,21.1,2.026526,20.6,0.0,11.329319,1019.000000,290.0,1.5,Winter


In [8]:
# Confirming we have imputed the values correctly  
determine_column_counts(weather_imputed)

,column_name,column_type,num_rows,distinct_data,distinct_percent,missing_data,missing_percentage
0,timestamp,datetime64[ns],331166,17544,5.298,0,0.0
1,site_id,object,331166,19,0.006,0,0.0
2,airTemperature,float64,331166,650,0.196,0,0.0
3,cloudCoverage,float64,331166,14,0.004,0,0.0
4,dewTemperature,float64,331166,557,0.168,0,0.0
5,precipDepth1HR,float64,331166,171,0.052,0,0.0
6,precipDepth6HR,float64,331166,172,0.052,0,0.0
7,seaLvlPressure,float64,331166,760,0.229,0,0.0
8,windDirection,float64,331166,63,0.019,0,0.0
9,windSpeed,float64,331166,77,0.023,0,0.0


In [9]:
# Aggregating weather daily 
weather_imputed['timestamp'] = pd.to_datetime(weather_imputed['timestamp'])
weather_imputed['date'] = weather_imputed['timestamp'].dt.date

numeric_cols = weather_imputed.select_dtypes(include=['number']).columns
non_numeric_cols = weather_imputed.select_dtypes(exclude=['number', 'datetime64']).columns

# Aggregate the numeric columns by day
numeric_daily_avg = weather_imputed.groupby('date')[numeric_cols].mean()

# Merge the aggregated numeric DataFrame with the non-numeric columns
daily_avg = pd.merge(numeric_daily_avg, weather_imputed[non_numeric_cols].drop_duplicates(), on='date')

In [10]:
daily_avg[daily_avg['site_id'] == 'Wolf']

,date,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed,site_id,season
14,2016-01-01,5.246861,1.927009,0.254484,0.351088,10.801125,1018.888301,172.924863,3.807399,Wolf,Winter
33,2016-01-02,5.993973,1.997893,0.892188,0.409453,11.105558,1014.347411,181.359441,4.202455,Wolf,Winter
52,2016-01-03,5.660314,1.946017,0.778475,0.552568,11.167389,1010.396019,208.978674,4.015919,Wolf,Winter
71,2016-01-04,5.048507,1.987616,-0.268905,0.479493,11.089874,1008.903334,211.377040,3.909701,Wolf,Winter
90,2016-01-05,4.745567,2.007311,0.321921,1.033857,11.723586,1012.747700,170.002007,3.528571,Wolf,Winter
...,...,...,...,...,...,...,...,...,...,...,...
13788,2017-12-27,0.148770,1.727177,-4.908725,0.319127,11.634468,1020.046240,217.730218,3.687248,Wolf,Winter
13807,2017-12-28,-0.844098,1.555937,-6.204677,0.334733,10.882579,1023.015464,220.044428,3.277060,Wolf,Winter
13826,2017-12-29,0.168282,2.037777,-5.045740,0.255286,10.993293,1018.470049,203.160684,3.687334,Wolf,Winter
13845,2017-12-30,1.567716,2.108643,-3.203027,0.233554,10.862294,1014.599862,197.386918,4.084761,Wolf,Winter


Checking to make sure we have the correct number of rows for each site id

In [11]:
# Combining with meter-meta-data
meter_meta = pd.read_csv("../data/cleaned/cleaned_meters_meta.csv", index_col=0)

In [12]:
#Convert 'date' columns to datetime
meter_meta['date'] = pd.to_datetime(meter_meta['date'])
daily_avg['date'] = pd.to_datetime(daily_avg['date'])

# Ensure 'site_id' columns are the same type
meter_meta['site_id'] = meter_meta['site_id'].astype(str)
daily_avg['site_id'] = daily_avg['site_id'].astype(str)

# Remove any potential leading/trailing spaces
meter_meta['site_id'] = meter_meta['site_id'].str.strip()
daily_avg['site_id'] = daily_avg['site_id'].str.strip()

# Perform the merge again
merged_df = pd.merge(meter_meta, daily_avg, on=['site_id', 'date'], how='left')

In [13]:
# Checking the merged data table for one building id
merged_df[merged_df['building_id_kaggle'] == 1049.0]

,building_id,meter,date,meter_reading,site_id,building_id_kaggle,site_id_kaggle,sub_primaryspaceusage,sqm,sqft,timezone,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed,season
892551,Wolf_education_Vivian,electricity,2016-01-01,1300.5050,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,5.246861,1.927009,0.254484,0.351088,10.801125,1018.888301,172.924863,3.807399,Winter
892552,Wolf_education_Vivian,electricity,2016-01-02,2146.6925,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,5.993973,1.997893,0.892188,0.409453,11.105558,1014.347411,181.359441,4.202455,Winter
892553,Wolf_education_Vivian,electricity,2016-01-03,2406.7400,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,5.660314,1.946017,0.778475,0.552568,11.167389,1010.396019,208.978674,4.015919,Winter
892554,Wolf_education_Vivian,electricity,2016-01-04,1969.4725,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,5.048507,1.987616,-0.268905,0.479493,11.089874,1008.903334,211.377040,3.909701,Winter
892555,Wolf_education_Vivian,electricity,2016-01-05,835.2575,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,4.745567,2.007311,0.321921,1.033857,11.723586,1012.747700,170.002007,3.528571,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894008,Wolf_education_Vivian,water,2017-12-27,2957.4056,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,0.148770,1.727177,-4.908725,0.319127,11.634468,1020.046240,217.730218,3.687248,Winter
894009,Wolf_education_Vivian,water,2017-12-28,1713.1557,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,-0.844098,1.555937,-6.204677,0.334733,10.882579,1023.015464,220.044428,3.277060,Winter
894010,Wolf_education_Vivian,water,2017-12-29,2110.7344,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,0.168282,2.037777,-5.045740,0.255286,10.993293,1018.470049,203.160684,3.687334,Winter
894011,Wolf_education_Vivian,water,2017-12-30,1809.5784,Wolf,1049.0,12.0,College Classroom,8770.0,94399.0,Europe/Dublin,1.567716,2.108643,-3.203027,0.233554,10.862294,1014.599862,197.386918,4.084761,Winter


In [14]:
merged_df.to_csv("../data/cleaned/cleaned_all_data.csv")